<h1>APS 4</h1>

In [1]:
"""
A funcao 'plota' produz um gráfico da estrutura definida pela matriz de nos N 
e pela incidencia Inc.

Sugestao de uso:

from funcoesTermosol import plota
plota(N,Inc)
-------------------------------------------------------------------------------
A funcao 'importa' retorna o numero de nos [nn], a matriz dos nos [N], o numero
de membros [nm], a matriz de incidencia [Inc], o numero de cargas [nc], o vetor
carregamento [F], o numero de restricoes [nr] e o vetor de restricoes [R] 
contidos no arquivo de entrada.

Sugestao de uso:
    
from funcoesTermosol import importa
[nn,N,nm,Inc,nc,F,nr,R] = importa('entrada.xlsx')
-------------------------------------------------------------------------------
A funcao 'geraSaida' cria um arquivo nome.txt contendo as reacoes de apoio Ft, 
deslocamentos Ut, deformacoes Epsi, forcas Fi e tensoes Ti internas. 
As entradas devem ser vetores coluna.

Sugestao de uso:
    
from funcoesTermosol import geraSaida
geraSaida(nome,Ft,Ut,Epsi,Fi,Ti)
-------------------------------------------------------------------------------

"""
import math
from traceback import print_list

import numpy as np


def plota(N,Inc):
    # Numero de membros
    nm = len(Inc[:,0])
    
    import matplotlib as mpl
    import matplotlib.pyplot as plt

#    plt.show()
    fig = plt.figure()
    # Passa por todos os membros
    for i in range(nm):
        
        # encontra no inicial [n1] e final [n2] 
        n1 = int(Inc[i,0])
        n2 = int(Inc[i,1])        

        plt.plot([N[0,n1-1],N[0,n2-1]],[N[1,n1-1],N[1,n2-1]],color='r',linewidth=3)


    plt.xlabel('x [m]')
    plt.ylabel('y [m]')
    plt.grid(True)
    plt.axis('equal')
    plt.show()
    
def importa(entradaNome):
    
    import numpy as np
    import xlrd
    
    arquivo = xlrd.open_workbook(entradaNome)
    
    ################################################## Ler os nos
    nos = arquivo.sheet_by_name('Nos')
    
    # Numero de nos
    nn = int(nos.cell(1,3).value)
                 
    # Matriz dos nós
    N = np.zeros((2,nn))
    
    for c in range(nn):
        N[0,c] = nos.cell(c+1,0).value
        N[1,c] = nos.cell(c+1,1).value
    
    ################################################## Ler a incidencia
    incid = arquivo.sheet_by_name('Incidencia')
    
    # Numero de membros
    nm = int(incid.cell(1,5).value)
                 
    # Matriz de incidencia
    Inc = np.zeros((nm,4))
    
    for c in range(nm):
        Inc[c,0] = int(incid.cell(c+1,0).value)
        Inc[c,1] = int(incid.cell(c+1,1).value)
        Inc[c,2] = incid.cell(c+1,2).value
        Inc[c,3] = incid.cell(c+1,3).value
    
    ################################################## Ler as cargas
    carg = arquivo.sheet_by_name('Carregamento')
    
    # Numero de cargas
    nc = int(carg.cell(1,4).value)
                 
    # Vetor carregamento
    F = np.zeros((nn*2,1))
    
    for c in range(nc):
        no = carg.cell(c+1,0).value
        xouy = carg.cell(c+1,1).value
        GDL = int(no*2-(2-xouy)) 
        F[GDL-1,0] = carg.cell(c+1,2).value
         
    ################################################## Ler restricoes
    restr = arquivo.sheet_by_name('Restricao')
    
    # Numero de restricoes
    nr = int(restr.cell(1,3).value)
                 
    # Vetor com os graus de liberdade restritos
    R = np.zeros((nr,1))
    
    for c in range(nr):
        no = restr.cell(c+1,0).value
        xouy = restr.cell(c+1,1).value
        GDL = no*2-(2-xouy) 
        R[c,0] = GDL-1


    return nn,N,nm,Inc,nc,F,nr,R

def geraSaida(nome,Ft,Ut,Epsi,Fi,Ti):
    nome = nome + '.txt'
    f = open("saida.txt","w+")
    f.write('Reacoes de apoio [N]\n')
    f.write(str(Ft))
    f.write('\n\nDeslocamentos [m]\n')
    f.write(str(Ut))
    f.write('\n\nDeformacoes []\n')
    f.write(str(Epsi))
    f.write('\n\nForcas internas [N]\n')
    f.write(str(Fi))
    f.write('\n\nTensoes internas [Pa]\n')
    f.write(str(Ti))
    f.close()
    


nn,N,nm,Inc,nc,F,nr,R = importa(entradaNome='entrada.xlsx')


def calculo_L_angulos(N, Inc):
    linhas, colunas = N.shape
    dict_nos = {}
    dict_l = {}
    for i in range(colunas):
        x = N[0][i]
        y = N[1][i]
        dict_nos[i+1] = [x, y]

    linhas_inc, colunas_inc = Inc.shape

    for j in range(linhas_inc):
        no1 = Inc[j][0]
        no2 = Inc[j][1]
        x1, y1 = dict_nos[no1]
        x2, y2 = dict_nos[no2]
        l = math.sqrt((x1-x2)**2+(y1-y2)**2)
        sen = (y2-y1)/l
        cos = (x2-x1)/l
        #comprimento, seno, cosseno, E, A, no1, no2
        dict_l[j+1] = [l, sen, cos, Inc[j][2], Inc[j][3], Inc[j][0], Inc[j][1]]
    
    return dict_l


def define_nos_graus_liberdade(dict_L):
    dict_nos_graus = {}
            
    for i in range(1, len(dict_L)+1):
        if (dict_L[i][5] not in dict_nos_graus):
            print(dict_L[i][5])
            dict_nos_graus[int(dict_L[i][5])] = 0
            
    j = 1;
    begin = len(dict_nos_graus)+1
    end = begin + len(dict_L)
    
    for i in range(begin, end): 
        print(dict_L[j][6])
        if (dict_L[j][6] not in dict_nos_graus):
            dict_nos_graus[int(dict_L[j][6])] = 0
        j+=1
        
    dict_nos_graus = dict(sorted(dict_nos_graus.items()))
    
    contador = 1

    for i in range(1,len(dict_nos_graus)+1):
        dict_nos_graus[i] = [contador , contador + 1]
        contador += 2
                    
    return dict_nos_graus



def calcula_Ke(dict_l):
    dict_Ke = {}
    dict_K_elemento = {}

    for elm in dict_l.keys():
        E = dict_l[elm][3]
        A = dict_l[elm][4]
        s = dict_l[elm][1]
        c = dict_l[elm][2]
        L = dict_l[elm][0]
        

        dict_Ke[elm] =  [[c**2, c*s, -c**2, -c*s],
                        [c*s, s**2, -c*s, -s**2],
                        [-c**2, -c*s, c**2, c*s],
                        [-c*s, -s**2, c*s, s**2]]

        k = ((E*A)/L)
        dict_K_elemento[elm] = np.multiply(dict_Ke[elm], k)

    return dict_K_elemento


def relaciona_elemento_graus_liberdade(dict_L, dict_nos_graus):
    elemento_graus_liberdade = {}

    for i in range(1, len(dict_L)+1):
        elemento_graus_liberdade[i] = []
        
    for i in range(1, len(dict_L)+1):
        for j in range(1, len(dict_nos_graus)+1):
            if (dict_L[i][5] == j): # se elemento nao está no dic e nó é o que quero ver os graus de lib
                elemento_graus_liberdade[i].extend(dict_nos_graus[j])
            elif (dict_L[i][6] == j):
                elemento_graus_liberdade[i].extend(dict_nos_graus[j]) 

    return elemento_graus_liberdade
    

def montaMatrizGeral(dict_K_elemento, elemento_graus_liberdade, dict_nos_graus):
    matrizGeral = np.zeros((len(dict_nos_graus)*2, len(dict_nos_graus)*2))
    
    for elemento in dict_K_elemento.keys():
        u1 = elemento_graus_liberdade[elemento][0]  #lembrando que index de matriz é isso menos 1, já que começa em 0
        u2 = elemento_graus_liberdade[elemento][1]
        u3 = elemento_graus_liberdade[elemento][2]
        u4 = elemento_graus_liberdade[elemento][3]
        
        matrizGeral[u1-1][u1-1] += dict_K_elemento[elemento][0][0]
        matrizGeral[u1-1][u2-1] += dict_K_elemento[elemento][0][1]
        matrizGeral[u1-1][u3-1] += dict_K_elemento[elemento][0][2]
        matrizGeral[u1-1][u4-1] += dict_K_elemento[elemento][0][3]
        
        matrizGeral[u2-1][u1-1] += dict_K_elemento[elemento][1][0]
        matrizGeral[u2-1][u2-1] += dict_K_elemento[elemento][1][1]
        matrizGeral[u2-1][u3-1] += dict_K_elemento[elemento][1][2]
        matrizGeral[u2-1][u4-1] += dict_K_elemento[elemento][1][3]
        
        matrizGeral[u3-1][u1-1] += dict_K_elemento[elemento][2][0]
        matrizGeral[u3-1][u2-1] += dict_K_elemento[elemento][2][1]
        matrizGeral[u3-1][u3-1] += dict_K_elemento[elemento][2][2]
        matrizGeral[u3-1][u4-1] += dict_K_elemento[elemento][2][3]
        
        matrizGeral[u4-1][u1-1] += dict_K_elemento[elemento][3][0]
        matrizGeral[u4-1][u2-1] += dict_K_elemento[elemento][3][1]
        matrizGeral[u4-1][u3-1] += dict_K_elemento[elemento][3][2]
        matrizGeral[u4-1][u4-1] += dict_K_elemento[elemento][3][3]

    return matrizGeral



def aplicaRestricao(R, matrizGeral, F):
    exclui_lista = []

    for restricao in R:
        exclui_lista.append(int(restricao[0]))
    
    exclui_lista.sort(reverse = True)

    for i in range(len(exclui_lista)):
        matrizGeral = np.delete(matrizGeral, exclui_lista[i], axis=0)    
        matrizGeral = np.delete(matrizGeral, exclui_lista[i], axis=1)    
        F = np.delete(F, exclui_lista[i], axis=0)
        
    return exclui_lista, matrizGeral, F


def resolve_gauss(ite, tol, K, F):
    contador = 0
    linhas, colunas = np.shape(K)
    X = np.zeros((linhas, 1))
    X_anterior = np.zeros((linhas, 1))
    erros_lista = np.ones((linhas, 1))
    
    while (contador < ite):
        for i in range(linhas):
            X_anterior[i] = X[i]   
        for i in range(linhas):
            soma_k_vezes_u = 0
            for j in range(colunas):
                if (j != i):
                    soma_k_vezes_u += K[i][j]*X[j]    

            resultado = (F[i]-soma_k_vezes_u)/K[i][i]
            X[i] = resultado

            if (X[i] != 0):
                erros_lista[i] = abs((X[i] - X_anterior[i])/X[i])

        erro = np.amax(erros_lista)

        contador+=1
        
        if erro <= tol :
            break

    return X


def matriz_deslocamento_completa(deslocamento_matriz, exclui_lista, dict_nos_graus):
    matriz_desloca_completa = np.zeros((len(dict_nos_graus)*2, 1))
        
    exclui_lista.sort(reverse = False)
                
    j = 0
    z = 0

    for i in range(len(matriz_desloca_completa)):  
        if z < len(exclui_lista) and exclui_lista[z] == i:
            matriz_desloca_completa[i] = 0
            z+=1
        else:
            matriz_desloca_completa[i] = deslocamento_matriz[j]
            j+=1

    return matriz_desloca_completa


def determina_tensao(matriz_desloca_completa, dict_L, dict_nos_graus):
    matriz_tensao = {}
            
    i = 0

    for elm in dict_L.keys():
        E = dict_L[elm][3]
        s = dict_L[elm][1]
        c = dict_L[elm][2]
        L = dict_L[elm][0]
        no1 = dict_L[elm][5]
        no2 = dict_L[elm][6]
        
        k = (E/L)
        matriz_angulos = [[-c, -s, c, s]]  
        
        index1 = (dict_nos_graus[int(no1)][0])-1
        index2 = (dict_nos_graus[int(no1)][1])-1
        index3 = (dict_nos_graus[int(no2)][0])-1
        index4 = (dict_nos_graus[int(no2)][1])-1
        
        matriz_u_vetor_nos = [[matriz_desloca_completa[index1][0]], 
                              [matriz_desloca_completa[index2][0]], 
                              [matriz_desloca_completa[index3][0]], 
                              [matriz_desloca_completa[index4][0]]]
        
        
        mult_matriz = np.matmul(matriz_angulos, matriz_u_vetor_nos)
            
        multiplica = np.multiply(mult_matriz, k)
        
        matriz_tensao[elm] = multiplica
        
        
        i+=2

    return matriz_tensao


def determina_deformacao(matriz_desloca_completa, dict_L, dict_nos_graus):
    matriz_def = {}
            
    i = 0

    for elm in dict_L.keys():
        s = dict_L[elm][1]
        c = dict_L[elm][2]
        L = dict_L[elm][0]
        no1 = dict_L[elm][5]
        no2 = dict_L[elm][6]
        
        k = (1/L)
        matriz_angulos = [[-c, -s, c, s]]  
        
        index1 = (dict_nos_graus[int(no1)][0])-1
        index2 = (dict_nos_graus[int(no1)][1])-1
        index3 = (dict_nos_graus[int(no2)][0])-1
        index4 = (dict_nos_graus[int(no2)][1])-1

        matriz_u_vetor_nos = [[matriz_desloca_completa[index1][0]], 
                              [matriz_desloca_completa[index2][0]], 
                              [matriz_desloca_completa[index3][0]], 
                              [matriz_desloca_completa[index4][0]]]
        
        mult_matriz = np.matmul(matriz_angulos, matriz_u_vetor_nos)
            
        multiplica = np.multiply(mult_matriz, k)
        
        matriz_def[elm] = multiplica
        
        
        i+=2

    return matriz_def


def determina_forca_reacao(F, matriz_desloca_completa, exclui_lista, matrizGeral):
    exclui_lista.sort(reverse = False)
    
    reacoes = np.dot(matrizGeral, matriz_desloca_completa)
    dict_reacoes = {}
    
    for i in range(len(exclui_lista)):
        for j in range(len(F)):
            if exclui_lista[i] == j:
                dict_reacoes[j+1] = reacoes[j]
    
    return dict_reacoes

def determina_forca_interna(matriz_tensao, dict_L):
    dict_forca_interna = {}
    for elm in dict_L:
        dict_forca_interna[elm] = matriz_tensao[elm]*dict_L[elm][4]
    
    return dict_forca_interna



dict_L = calculo_L_angulos(N, Inc)

dict_nos_graus = define_nos_graus_liberdade(dict_L)

dict_cada_elemento = calcula_Ke(dict_L)

dict_elemento_graus = relaciona_elemento_graus_liberdade(dict_L, dict_nos_graus)

matrizGeral = montaMatrizGeral(dict_cada_elemento, dict_elemento_graus, dict_nos_graus)

print(f'MATRIZ GERAL {matrizGeral}')

exclui_lista, matrizRestricao, Fajustado = aplicaRestricao(R, matrizGeral, F)

lista_resultado = resolve_gauss(10000, 1e-8, matrizRestricao, Fajustado)

print(f' MATRIZ RESTRIÇÃO {matrizRestricao}\n')


matriz_desloca_completa = matriz_deslocamento_completa(lista_resultado, exclui_lista, dict_nos_graus)    


# ----------------------------------------- As matrizes que realmente interessam ---------------------------------------

dict_tensao = determina_tensao(matriz_desloca_completa, dict_L, dict_nos_graus)
dataMatrixTensao = np.array([dict_tensao[i][0] for i in dict_tensao.keys()])

dict_deformacao = determina_deformacao(matriz_desloca_completa, dict_L, dict_nos_graus)
dataMatrixDeformacao = np.array([dict_deformacao[i][0] for i in dict_deformacao.keys()])

dict_reacoes = determina_forca_reacao(F, matriz_desloca_completa, exclui_lista, matrizGeral)
dataMatrixReacao = np.array([dict_reacoes[i] for i in dict_reacoes.keys()])

dict_forca_interna = determina_forca_interna(dict_tensao, dict_L)

dataMatrixInterna = np.array([dict_forca_interna[i][0] for i in dict_forca_interna.keys()])


print(f'Essa é o dicionário de tensão: {dict_tensao}\n')
print(f'Esse é o dicionário de deformação: {dict_deformacao}\n')
print(f'Esse é o dicionário de reacoes: {dict_reacoes}\n')
print(f'Esse é o dicionário de forças internas: {dict_forca_interna}\n')

print("---------------------------------------------------------------------------------------------------------------")

print(f'Essa é a matriz de deslocamento: {matriz_desloca_completa}\n')
print(f'Essa é a matriz de tensão: {dataMatrixTensao}\n')
print(f'Essa é a matriz de deformação: {dataMatrixDeformacao}\n')
print(f'Essa é a matriz de reações: {dataMatrixReacao}\n')
print(f'Essa é a matriz de forças internas: {dataMatrixInterna}\n')


geraSaida("resultados", dataMatrixReacao,matriz_desloca_completa,dataMatrixDeformacao,dataMatrixInterna,dataMatrixTensao)

plota(N,Inc)



AttributeError: 'ElementTree' object has no attribute 'getiterator'